In [21]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import textblob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# TFIDF (term freq- inverse document feq)
# countvectorizer 


# for import 
# nltk
# textblob 

In [22]:
df = pd.read_csv(r'D:\My Files\Python_001\AI and ML Feb 2024\Unsupervised_learning\imdb_clean.csv')
df.head(5)

,Unnamed: 0,title,director,release_year,runtime,genre,rating,metascore,gross(M)
0,0,The Shawshank Redemption,Frank Darabont,1994,142,Drama,9.3,82,28.34
1,1,The Godfather,Francis Ford Coppola,1972,175,Crime,9.2,100,134.97
2,1,The Godfather,Francis Ford Coppola,1972,175,Drama,9.2,100,134.97
3,2,The Dark Knight,Christopher Nolan,2008,152,Action,9.0,84,534.86
4,2,The Dark Knight,Christopher Nolan,2008,152,Crime,9.0,84,534.86


In [23]:
df['data'] = df['title'] + ' ' + df['director'] + ' ' + df['genre']
df['data']

0       The Shawshank Redemption Frank Darabont Drama
1            The Godfather Francis Ford Coppola Crime
2            The Godfather Francis Ford Coppola Drama
3            The Dark Knight Christopher Nolan Action
4             The Dark Knight Christopher Nolan Crime
                            ...                      
2527             The Invisible Man James Whale Horror
2528             The Invisible Man James Whale Sci-Fi
2529                    Cell 211 Daniel Monzón Action
2530                     Cell 211 Daniel Monzón Crime
2531                     Cell 211 Daniel Monzón Drama
Name: data, Length: 2532, dtype: object

In [24]:
# remive punctuation 

df['data'] = df['data'].str.replace('[^\w\s]','')
#lower case
df['data'] = df['data'].str.lower()

In [25]:
df['data']

0       the shawshank redemption frank darabont drama
1            the godfather francis ford coppola crime
2            the godfather francis ford coppola drama
3            the dark knight christopher nolan action
4             the dark knight christopher nolan crime
                            ...                      
2527             the invisible man james whale horror
2528             the invisible man james whale sci-fi
2529                    cell 211 daniel monzón action
2530                     cell 211 daniel monzón crime
2531                     cell 211 daniel monzón drama
Name: data, Length: 2532, dtype: object

In [26]:
def remove_stopwords (text):
    words = text.split()
    return " ".join([word for word in words if word not in stopwords.words('english')])
df['data'] = df['data'].apply(remove_stopwords)
df['data']

0       shawshank redemption frank darabont drama
1            godfather francis ford coppola crime
2            godfather francis ford coppola drama
3            dark knight christopher nolan action
4             dark knight christopher nolan crime
                          ...                    
2527             invisible man james whale horror
2528             invisible man james whale sci-fi
2529                cell 211 daniel monzón action
2530                 cell 211 daniel monzón crime
2531                 cell 211 daniel monzón drama
Name: data, Length: 2532, dtype: object

In [27]:
# for revome error (error; stopword not found)
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\HP
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
# vectorize the data 
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['data']).toarray()
X 


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [29]:
X.shape

(2532, 2267)

In [30]:
# similarity Matrix 
similarity = cosine_similarity(X, X)
similarity

array([[1. , 0. , 0.2, ..., 0. , 0. , 0.2],
       [0. , 1. , 0.8, ..., 0. , 0.2, 0. ],
       [0.2, 0.8, 1. , ..., 0. , 0. , 0.2],
       ...,
       [0. , 0. , 0. , ..., 1. , 0.8, 0.8],
       [0. , 0.2, 0. , ..., 0.8, 1. , 0.8],
       [0.2, 0. , 0.2, ..., 0.8, 0.8, 1. ]])

In [31]:
similarity.shape


(2532, 2532)

In [34]:
def get_index_from_title(title):
    try:
        return df[df.title == title].index[0]
    except:
        return None
    
def recommend_movie(title, limit =10):
    index = get_index_from_title(title)
    if index is None:
        return []
    else:
        movie_scores = []
        for i in range(similarity.shape[0]):
            movie_scores.append((df['title'][i], similarity[index][i]))
        movie_scores.sort(key= lambda X: X[1], reverse=True)
        return movie_scores[1:limit + 1]

In [35]:
recommend_movie('Harry Potter and the Goblet of Fire', 5)

[('Harry Potter and the Goblet of Fire', 0.8571428571428569),
 ('Harry Potter and the Goblet of Fire', 0.8571428571428569),
 ('Harry Potter and the Prisoner of Azkaban', 0.4285714285714285),
 ("Harry Potter and the Sorcerer's Stone", 0.4285714285714285),
 ('Harry Potter and the Deathly Hallows: Part 2', 0.40089186286863654)]